# Identify Unknown images

## Setup the environment

In [ ]:
# nuclio: ignore
import nuclio
import os

### Set environment variables

In [ ]:
# nuclio: ignore
os.environ['KV_TABLE'] = os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'demos/face-recognition/artifacts/encodings')

In [ ]:
# Iguazio access
%nuclio env V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env KV_TABLE=${KV_TABLE}


### Set cron trigger

In [ ]:
%nuclio config spec.triggers.secs.kind = "cron"
%nuclio config spec.triggers.secs.attributes.interval = "1h"



### Set iguazio mount

In [ ]:
%nuclio mount /User users/iguazio

In [ ]:
### Base image

In [ ]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

### Installations

When installing packages while working, Please reset the kernel to allow Jupyter to load the new packages.

In [ ]:
%%nuclio cmd -c
pip install v3io_frames

### Imports

In [ ]:
# Util
import v3io_frames as v3f
from datetime import datetime, timedelta
import os
import shutil

# DB
import v3io_frames as v3f

## Function code

In [ ]:
def handler(context, event):
    kv_table_path = os.environ['KV_TABLE']
    v3io_access_key = os.environ['V3IO_ACCESS_KEY']
    client = v3f.Client("framesd:8081", token = v3io_access_key, container="users") 
    df = client.read(backend='kv', table=kv_table_path, reset_index=True) 
    context.logger.info(df.head())  
    df2 = df[['fileName', 'camera', 'label', 'imgUrl']]    
    options = ['unknown']
    df3 = df2[df2.fileName.str.startswith(tuple(options))]    
    for idx in range(len(df3)):
        img_url = df3.iloc[idx]['imgUrl']
        splited = img_url.split("/")
        destination = "/".join((splited[0],splited[1],splited[2],splited[3],"dataset/label_pending"))
        print(img_url)
        print(idx)
        print(splited)
        print(destination)
        # Move the content of
        # source to destination
        dest = shutil.move(img_url, destination)

In [ ]:
%nuclio deploy -n unknown-labeling -p default -c